In [ ]:
#imports
import re
import os
import pandas as pd
from pprint import pprint

In [ ]:
# process each row function
def func(row):
    row = re.sub(r'\s{3,}', '\t', row).strip().split('\t')
    if len(row)==1:     #to remove empty row by putting None for that position
        if row[0]=='':
            return None
    elif len(row)==2:   #to removes row with only 2 columns filled (member total row gets removed) - by placing None as value for that row
        return None
    elif len(row)==5:   #extract the amount column from row with memberid/name and place it in 14th column
        bal = row[-3]
        row[-3] = '#'
        row = row + ['']*8
        row.append(bal)
    else:               #extract the amount,user column from all remaining rows place it in 14th/15th column
        bal = row[-2]
        user = row[-1]
        row[-2] = '#'
        row[-1] = '#'
        row = row + [''] * (13-len(row))
        row.append(bal)
        row.append(user)
    return row


In [ ]:
import os
notebook_path = os.path.abspath("pdf_text_extractor.ipynb")
print(notebook_path)
work_dir = os.path.dirname(notebook_path)

In [ ]:
# define the input filepath
file = os.path.join(work_dir,'DataWindow.txt')

# read input file
with open(file) as filetext:
    file_content_list = filetext.readlines()
pprint(file_content_list[:20])

In [ ]:
# applying the func function to each row
conv_content_list = list(map(func, file_content_list))

# remove rows with None
conv_content_list = [val for val in conv_content_list if val]
pprint(conv_content_list[:6])

In [ ]:
'''Clean up'''
# creating page break and the column info rows list
page_break = []
doctor_name =[]
for idx,row in enumerate(conv_content_list[7:]): #conv_content_list[7:] to remove column names etc from begininng
    if row[0][:28]=='----------------------- Page':
        idx_list = list(range(idx,idx+7,1))
        page_break = page_break + idx_list
    elif row[0].strip() =='*Bold indicates transaction has been deleted or voided':
        page_break.append(idx)
    if '.0-' in row[0]:
        doctor_name.append(idx)
        
pprint(f'page_break row list (showing limited list):{page_break[:20]}')
pprint(f'member name row list (showing limited list):{doctor_name[:20]}')

In [ ]:
# creating member index list for transactions for belonging to same member
index_key = []
for i,idx in enumerate(doctor_name):
    try:
        idx_list = [idx]*(doctor_name[i+1]-idx)
    except IndexError:
        idx_list = [idx]*(len(conv_content_list[7:])-idx)
    index_key.extend(idx_list)

In [ ]:
# create pandas table (dateframe) from list of rows
df = pd.DataFrame(conv_content_list[7:])        #conv_content_list[7:] to remove column names etc from begininng

# fill empty cells with np.nan
df.fillna(value=pd.np.nan, inplace=True)
# df['idx'] = df.count(axis=1)

# creating index column
df = df.reset_index()

''' more cleanup and data massaging'''
# creating the member index 
df['index_key'] = index_key

# removing the end of file summary rows as they don't conform to the transaction columns
# change the row number based on the file.
# print(df.iloc[3580:3600])
df = df[:3592]

# removing the page break etc rows
df = df[~df['index'].isin(page_break)]

# setting index based on member id index list column
df = df.set_index('index_key')
df_tmp = df.drop(columns=['index'])

print('********Head********\n',df.head())
print('********Tail********\n',df.tail())

In [ ]:
'''to get memberid/name for each transaction row'''
# splitting df for memberid/name rows and removing duplicates due to parsing errors.
df_name = df_tmp[df_tmp[14].isna()]
df_name = df_name.loc[~df_name.index.duplicated(keep='first')][[0,1]]

# df for transaction rows
df_trans = df_tmp[~df_tmp[14].isna()]

# merging the name & transaction df to get memberid/name for each transaction row (uses member id indexes)
new_df = pd.merge(df_name,df_trans,how='outer',left_index=True,right_index=True)
print(new_df.head())

In [ ]:
# outputing the csv file
output_file = os.path.join(work_dir,'output.csv')
new_df.to_csv(output_file)

print('Output file:',output_file)
